In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from scipy.stats import friedmanchisquare
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import friedmanchisquare
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests
from scipy.stats import wilcoxon
from itertools import combinations
import openpyxl
import scienceplots

In [2]:
#Load Questionairs (in flight)
# se base foler

# Define the base folder path
#base_folder = r'C:\Users\ejnar\Desktop\Special sync\Special'
base_folder = r'C:\Users\ejnar\OneDrive\DTU\speciale_projekt\Spirometer fra test'
# Define the path to the Excel file
excel_file_path1 = os.path.join(base_folder, 'BestTestExport.xlsx')

# Load the Excel file into a DataFrame
df = pd.read_excel(excel_file_path1)

In [3]:
# Specify the columns you want to keep
columns_to_keep = ['SESSION_DATE_TIME','SUBJECT_ID','Medical_Test','FEV1',
                   'FVC','PEF','FEF2575','FEV1/FVC'
                  ]

# Keep only the specified columns
df = df[columns_to_keep]

In [4]:
# Ensure column name is correct
column_name = 'FEV1/FVC'

# Multiply the column values by 100
df[column_name] = df[column_name] * 100


In [5]:
#Renameing of collums 
df.rename(columns={'FEV1': 'Forced Expiratory Volume', 'FVC': 'Forced Vital Capacity', 'PEF': 'Peak Expiratory Flow',
                   'FEF2575': 'Forced Expiratory Flow 25-75'
                }, inplace=True)

In [6]:
#all 4 times filter
participants_to_keep = [1, 2, 4, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 
                        30, 31, 32, 34, 35, 36, 37, 38, 39, 41, 42, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 
                        57, 58, 59, 61, 63, 64, 65, 66, 67, 68, 69]

# Filter the DataFrame
df = df[df['SUBJECT_ID'].isin(participants_to_keep)]

In [7]:
# Convert SESSION_DATE_TIME to datetime format
df['SESSION_DATE_TIME'] = pd.to_datetime(df['SESSION_DATE_TIME'], format='%Y%m%d %H:%M:%S')

# Filter the rows based on the date range
start_date = pd.to_datetime('20240902', format='%Y%m%d')
end_date = pd.to_datetime('20240928', format='%Y%m%d')

df = df[(df['SESSION_DATE_TIME'] >= start_date) & (df['SESSION_DATE_TIME'] <= end_date)]


In [8]:
# Function to determine Medical_Test based on time
def assign_medical_test(session_time):
    time = session_time.time()
    if pd.Timestamp('09:00').time() <= time <= pd.Timestamp('10:30').time():
        return 1
    elif pd.Timestamp('14:30').time() <= time <= pd.Timestamp('17:00').time():
        return 2
    elif pd.Timestamp('18:00').time() <= time <= pd.Timestamp('21:00').time():
        return 3
    else:
        return None  # You can use 0 or another value for times outside these ranges

# Apply the function to create Medical_Test column
df['Medical_Test'] = df['SESSION_DATE_TIME'].apply(assign_medical_test)

In [9]:
#set filter type
# Convert SESSION_DATE_TIME to datetime
df["SESSION_DATE_TIME"] = pd.to_datetime(df["SESSION_DATE_TIME"], format="%Y%m%d %H:%M:%S")

# Define a function to assign A, B, C, D based on the date
def assign_filter(date_time):
    if date_time.day in [3, 13, 19, 25]:  # Check if day is in the list
        return "A"
    elif date_time.day in [4, 10, 20, 26]:  # Check if day is in the list
        return "B"
    elif date_time.day in [5, 11, 17, 27]:  # Check if day is in the list
        return "C"
    elif date_time.day in [6, 12, 18, 24]:  # Check if day is in the list
        return "D"
    else:
        return "Error"  # Return 'Error' if no match

# Apply the function to the filter column
df["filter"] = df["SESSION_DATE_TIME"].apply(assign_filter)                                       

In [10]:
#divided into filter
dfHepa7 = df[df['filter'] == 'A']
dfAPSU = df[df['filter'] == 'B']
dfHepa5 = df[df['filter'] == 'C']
dfAPSN = df[df['filter'] == 'D']

In [11]:
# Get the current working directory
current_directory = os.getcwd()

# File name and path
file_name = "Full_spyro_output.xlsx"
file_path = os.path.join(current_directory, file_name)

# Save the DataFrame to an Excel file
df.to_excel(file_path, index=False, engine='openpyxl')

In [12]:
#manualt change
#name of boxplot

#nametag change
nametag = "All"
#nametag = "Men"
#nametag = "Women"
#nametag = "Young"
#nametag = "Juniors"
#nametag = "Elderly"
#nametag = "Seniors"
#nametag = "Sensitive"
#nametag = "Non-Sensitive"
name_suffix = nametag

In [13]:
#person filter
#manualt change
Participant_number_men = [1, 3, 7, 10, 11, 12, 16, 18, 19, 22, 24, 26, 27, 30, 31, 32, 33, 34, 37, 39, 42, 44, 46, 47, 49, 50, 51, 53, 55, 59, 61, 62, 63, 64, 65, 67, 68]

Participant_number_women = [2, 4, 5, 6, 8, 9, 13, 14, 15, 17, 20, 21, 23, 25, 28, 29, 35, 36, 38, 40, 41, 43, 45, 52, 54, 56, 57, 58, 60, 66, 69]

Participant_number_young = [1, 3, 4, 5, 6, 9, 13, 14, 15, 16, 18, 19, 20, 21, 26, 29, 30, 32, 33, 34, 37, 38, 39, 43, 45, 46, 47, 49, 51, 52, 55, 57, 59, 60, 61, 62, 65, 67, 68]

Participant_number_old = [ 2, 7, 8, 10, 11, 12, 17, 22, 23, 24, 25, 27, 28, 31, 35, 36, 40, 41, 42, 44, 50, 53, 54, 56, 58, 63, 64, 66, 69]

Participant_number_sen = [34, 35, 65, 39, 19, 4, 22, 6, 7, 9, 47, 10, 31, 32, 15, 17, 37]

Participant_number_nonsen = [1, 52, 2, 36, 18, 53, 38, 20, 54, 21, 56, 41, 23, 55, 57, 58, 24, 42, 26, 59, 44, 27, 8, 69, 28, 29, 30, 61, 45, 46, 11, 63, 64, 49, 13, 14, 66, 16, 51, 67, 50, 68]

# Filter the DataFrame
#dfHepa7 = dfHepa7[dfHepa7['SUBJECT_ID'].isin(Participant_number_nonsen)]
#dfHepa5 = dfHepa5[dfHepa5['SUBJECT_ID'].isin(Participant_number_nonsen)]
#dfAPSU = dfAPSU[dfAPSU['SUBJECT_ID'].isin(Participant_number_nonsen)]
#dfAPSN = dfAPSN[dfAPSN['SUBJECT_ID'].isin(Participant_number_nonsen)]

In [14]:
#boxplot of medical test
def sanitize_filename(name):
    """Replace invalid characters in a filename with underscores."""
    return re.sub(r'[<>:"/\\|?*]', '_', name)

def create_combined_boxplots(dataframes, labels, columns_to_plot, name_suffix="", nametag="", y_min=0, y_max=102):
    """Create combined box plots for specified columns in a list of DataFrames."""
    # Get the current directory where the script is placed
    current_directory = os.getcwd()
    output_folder = os.path.join(current_directory, "figur")

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Apply the 'science' style and 'no-latex'
    plt.style.use(['science', 'no-latex'])

    # Define the color palette to match the second code
    colors = ['#77AADD', '#99DDFF', '#44BB99', '#BBCC33']
    sns.set_palette(colors)

    # Assuming all DataFrames have the same 'Medical-Test' values
    medical_tests = dataframes[0]['Medical_Test'].unique()

    # Define custom x-tick labels
    custom_labels = ["MT1", "MT2", "MT3"]

    # Loop through the specified columns in the DataFrames
    for column in columns_to_plot:
        if column in dataframes[0].columns and pd.api.types.is_numeric_dtype(dataframes[0][column]):  # Check if column is numeric
            plt.figure(figsize=(10, 7))  # Match figure size

            # Combine data for seaborn boxplot
            combined_data = pd.concat([
                dataframe.assign(Dataset=labels[i]) 
                for i, dataframe in enumerate(dataframes)
            ])

            # Create the box plot
            sns.boxplot(
                x='Medical_Test', y=column, hue='Dataset', data=combined_data,
                palette=colors, dodge=True
            )

            # Add titles and labels
            plt.title(f"{column} - {nametag}",
                  fontsize=20, pad=15)
            plt.xlabel('Medical Test', fontsize=18)
            plt.ylabel('Ratio [-]', fontsize=18)

            # Set y-axis limits
            plt.ylim(y_min, y_max)

            # Set x-ticks and their labels
            plt.xticks(ticks=range(len(custom_labels)), labels=custom_labels, fontsize=16)
            plt.yticks(fontsize=16)

            # Add legend styled like the second code
            plt.legend(
                title="Conditions", title_fontsize=16, fontsize=14,
                loc='center left', bbox_to_anchor=(1, 0.5), frameon=False
            )

            # Add grid lines for y-axis
            plt.grid(visible=True, linestyle='--', linewidth=0.5, alpha=0.75, axis='y')

            # Adjust layout for better spacing
            plt.tight_layout()

            # Save the plot
            plot_filename = f"{sanitize_filename(column)}_{sanitize_filename(name_suffix)}.png"
            plt.savefig(os.path.join(output_folder, plot_filename), bbox_inches='tight')
            plt.close()

In [15]:
#først plot som er raw data 
dataframesRaw = [dfHepa7, dfHepa5, dfAPSN, dfAPSU]  


labels = ['HEPA 7', 'HEPA 5', 'APS N', 'APS U']  # Labels for each DataFrame

output_folder = base_folder

columns_to_plot1 = ['Forced Expiratory Volume','Forced Vital Capacity','Forced Expiratory Flow 25-75','Peak Expiratory Flow']
columns_to_plot2 = ['Peak Expiratory Flow']
# Create the combined boxplots with a name suffix and custom colors
create_combined_boxplots(dataframesRaw, labels,columns_to_plot1, name_suffix=name_suffix,nametag=nametag, y_min=0, y_max=10)
create_combined_boxplots(dataframesRaw, labels,columns_to_plot2, name_suffix=name_suffix,nametag=nametag, y_min=0, y_max=1000)

In [16]:
#plot
columns_to_plot4 = ['FEV1/FVC']
create_combined_boxplots(dataframesRaw, labels,columns_to_plot4, name_suffix=name_suffix,nametag=nametag, y_min=0, y_max=100)

In [17]:
# Define output file path
#Freidman and willcoxon by questionair
output_path = f'Friedman-Wilcoxon-by-Questinair-spyrometer-{name_suffix}.xlsx'

# DataFrames for each dataset
dataframes = {
    "Hepa 7": dfHepa7,
    "Hepa 5": dfHepa5,
    "APSU": dfAPSU,
    "APSN": dfAPSN
}

# Specify columns not to test
columns_not_to_test = ['SUBJECT_ID', 'Medical_Test', 'SESSION_DATE_TIME', 'filter']

# Alpha significance level
alpha = 0.1

# Initialize an Excel writer
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for dataset_name, df in dataframes.items():
        # Determine columns to test by excluding the specified ones
        columns_to_test = df.columns.difference(columns_not_to_test).tolist()

        # Initialize DataFrames to store results for the current dataset
        friedman_results_df = pd.DataFrame(columns=["Column", "Statistic", "p-value", "Alpha", "Significant", "<0.05", "<0.01", "<0.001"])
        wilcoxon_results_list = []

        # Loop through each column
        for col in columns_to_test:
            # Filter out data for each "Medical_Test" value (1 to 3) for the Friedman test
            medical_test_groups = [df[df["Medical_Test"] == i][col].dropna().values for i in range(1, 4)]
            
            # Perform the Friedman test
            friedman_stat, friedman_p = friedmanchisquare(*medical_test_groups)
            
            # Determine if the result is significant
            is_significant = 1 if friedman_p < alpha else 0
            
            # Check for p-value thresholds for Friedman test
            p_less_0_05 = 1 if friedman_p < 0.05 else 0
            p_less_0_01 = 1 if friedman_p < 0.01 else 0
            p_less_0_001 = 1 if friedman_p < 0.001 else 0

            # Append Friedman test results to the DataFrame
            temp_df = pd.DataFrame({
                "Column": [col],
                "Statistic": [friedman_stat],
                "p-value": [friedman_p],
                "Alpha": [alpha],
                "Significant": [is_significant],
                "<0.05": [p_less_0_05],
                "<0.01": [p_less_0_01],
                "<0.001": [p_less_0_001]
            })
            friedman_results_df = pd.concat([friedman_results_df, temp_df], ignore_index=True)

            # If significant
            if is_significant:
                # Get all pairs of Medical_Test
                pairs = combinations(range(1, 4), 2)
                for (i, j) in pairs:
                    # Get data for each pair
                    group1 = df[df["Medical_Test"] == i][col].dropna().values
                    group2 = df[df["Medical_Test"] == j][col].dropna().values
                    
                    
                    if len(group1) > 0 and len(group2) > 0:
                        wilcox_stat, wilcox_p = wilcoxon(group1, group2)
                        wilcoxon_significant = 1 if wilcox_p < alpha else 0 
                        
                        # Check for p-value thresholds for Wilcoxon test
                        wilcox_p_less_0_05 = 1 if wilcox_p < 0.05 else 0
                        wilcox_p_less_0_01 = 1 if wilcox_p < 0.01 else 0
                        wilcox_p_less_0_001 = 1 if wilcox_p < 0.001 else 0

                        # Calculate mean and average for both groups
                        mean_group1 = group1.mean()
                        mean_group2 = group2.mean()
                        avg_group1 = group1.mean()  
                        avg_group2 = group2.mean() 

                        # Determine which group is high
                        if mean_group1 > mean_group2 and avg_group1 > avg_group2:
                            mark = 1  
                        elif mean_group2 > mean_group1 and avg_group2 > avg_group1:
                            mark = -1  
                        else:
                            mark = 0  

                        # Append results to the Wilcoxon results list
                        wilcoxon_results_list.append({
                            "Column": col,
                            "Pair": f"{i}-{j}",
                            "Statistic": wilcox_stat,
                            "p-value": wilcox_p,
                            "Alpha": alpha,
                            "Significant": wilcoxon_significant,
                            "<0.05": wilcox_p_less_0_05,
                            "<0.01": wilcox_p_less_0_01,
                            "<0.001": wilcox_p_less_0_001,
                            "Mark": mark  
                        })

        # Save result
        friedman_results_df.to_excel(writer, sheet_name=f"{dataset_name} Friedman Results", index=False)
        
        if wilcoxon_results_list:
            wilcoxon_df = pd.DataFrame(wilcoxon_results_list)
            wilcoxon_df.to_excel(writer, sheet_name=f"{dataset_name} Wilcoxon Results", index=False)

print(f"Friedman and Wilcoxon test results saved to {output_path}.")


C:\Users\ejnar\AppData\Local\Temp\ipykernel_24968\4111221779.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  friedman_results_df = pd.concat([friedman_results_df, temp_df], ignore_index=True)
C:\Users\ejnar\AppData\Local\Temp\ipykernel_24968\4111221779.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  friedman_results_df = pd.concat([friedman_results_df, temp_df], ignore_index=True)
C:\Users\ejnar\AppData\Local\Temp\ipykernel_24968\4111221779.py:56: FutureWarning: The behavior of DataFra

Friedman and Wilcoxon test results saved to Friedman-Wilcoxon-by-Questinair-spyrometer-All.xlsx.


In [18]:
# Define output file path
#Freidman and willcoxon by conditions
output_path = f'Friedman-Wilcoxon-by-Filter-spyrometer-{name_suffix}.xlsx'

# DataFrames for each dataset
dataframes = {
    "Hepa 7": dfHepa7,
    "Hepa 5": dfHepa5,
    "APSU": dfAPSU,
    "APSN": dfAPSN
}

# Specify columns not to test
columns_not_to_test = ['SUBJECT_ID', 'Medical_Test', 'SESSION_DATE_TIME', 'filter']

# Alpha significance level
alpha = 0.1

# Initialize an Excel writer
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for medical_test in [1, 2, 3]:
        friedman_results_df = pd.DataFrame(columns=["Column", "Statistic", "p-value", "Alpha", "Significant", "<0.05", "<0.01", "<0.001"])
        wilcoxon_results_list = []  # Reset the Wilcoxon results list for each Medical_Test

        # Determine columns to test by excluding the specified ones
        columns_to_test = dataframes["Hepa 7"].columns.difference(columns_not_to_test).tolist()

        # Loop through each column
        for col in columns_to_test:
            # Collect data across datasets for the current column and Medical_Test
            data_groups = [
                dataframes[name][dataframes[name]["Medical_Test"] == medical_test][col].dropna().values
                for name in ["Hepa 7", "Hepa 5", "APSU", "APSN"]
            ]

            # Perform the Friedman test
            friedman_stat, friedman_p = friedmanchisquare(*data_groups)

            # Determine if the result is significant
            is_significant = 1 if friedman_p < alpha else 0

            # Check for p-value thresholds
            p_less_0_05 = 1 if friedman_p < 0.05 else 0
            p_less_0_01 = 1 if friedman_p < 0.01 else 0
            p_less_0_001 = 1 if friedman_p < 0.001 else 0

            # Append Friedman test results
            friedman_results_df = pd.concat([
                friedman_results_df,
                pd.DataFrame({
                    "Column": [col],
                    "Statistic": [friedman_stat],
                    "p-value": [friedman_p],
                    "Alpha": [alpha],
                    "Significant": [is_significant],
                    "<0.05": [p_less_0_05],
                    "<0.01": [p_less_0_01],
                    "<0.001": [p_less_0_001]
                })
            ], ignore_index=True)

            # If significant
            if is_significant:
                pairs = combinations(["Hepa 7", "Hepa 5", "APSU", "APSN"], 2)
                for (name1, name2) in pairs:
                    group1 = dataframes[name1][dataframes[name1]["Medical_Test"] == medical_test][col].dropna().values
                    group2 = dataframes[name2][dataframes[name2]["Medical_Test"] == medical_test][col].dropna().values

                    # Ensure both groups have data
                    if len(group1) > 0 and len(group2) > 0:
                        wilcox_stat, wilcox_p = wilcoxon(group1, group2)
                        wilcoxon_significant = 1 if wilcox_p < alpha else 0

                        # Calculate mean and average for both groups
                        mean_group1 = group1.mean()
                        mean_group2 = group2.mean()
                        avg_group1 = group1.mean()  
                        avg_group2 = group2.mean()  

                        # Determine which group is high
                        if mean_group1 > mean_group2 and avg_group1 > avg_group2:
                            mark = 1  # Variable 1 is higher
                        elif mean_group2 > mean_group1 and avg_group2 > avg_group1:
                            mark = -1  # Variable 2 is higher
                        else:
                            mark = 0  # Relationship is unknown

                        # Check for p-value thresholds for Wilcoxon
                        wilcox_p_less_0_05 = 1 if wilcox_p < 0.05 else 0
                        wilcox_p_less_0_01 = 1 if wilcox_p < 0.01 else 0
                        wilcox_p_less_0_001 = 1 if wilcox_p < 0.001 else 0

                        # Append results to the Wilcoxon results list
                        wilcoxon_results_list.append({
                            "Column": col,
                            "Pair": f"{name1} vs {name2}",
                            "Statistic": wilcox_stat,
                            "p-value": wilcox_p,
                            "Alpha": alpha,
                            "Significant": wilcoxon_significant,
                            "Mark": mark,  
                            "<0.05": wilcox_p_less_0_05,
                            "<0.01": wilcox_p_less_0_01,
                            "<0.001": wilcox_p_less_0_001
                        })


        # Save result
        friedman_results_df.to_excel(writer, sheet_name=f"Medical {medical_test} Friedman Results", index=False)

        if wilcoxon_results_list:
            wilcoxon_df = pd.DataFrame(wilcoxon_results_list)
            wilcoxon_df.to_excel(writer, sheet_name=f"Medical {medical_test} Wilcoxon Results", index=False)

print(f"Friedman and Wilcoxon test results saved to {output_path}.")


C:\Users\ejnar\AppData\Local\Temp\ipykernel_24968\694742594.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  friedman_results_df = pd.concat([
C:\Users\ejnar\AppData\Local\Temp\ipykernel_24968\694742594.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  friedman_results_df = pd.concat([
C:\Users\ejnar\AppData\Local\Temp\ipykernel_24968\694742594.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer

Friedman and Wilcoxon test results saved to Friedman-Wilcoxon-by-Filter-spyrometer-All.xlsx.


In [19]:
#below 0.7
#Keep result of below 0.7 in FEV1/FVC
results = []

# Loop through each DataFrame in dataframes
for name, df in dataframes.items():
    # Group by "Medical_Test" and calculate counts for "FEV1/FVC" > 70 and <= 70
    summary = df.groupby("Medical_Test")["FEV1/FVC"].agg(
        above_70=lambda x: (x > 70).sum(),
        below_or_equal_70=lambda x: (x <= 70).sum()
    ).reset_index()
    
    # Add dataset name for context
    summary["Dataset"] = name
    
    # Append to results
    results.append(summary)

# Concatenate results into a single DataFrame
final_df = pd.concat(results, ignore_index=True)

# Rearrange columns for better clarity
final_df = final_df[["Dataset", "Medical_Test", "above_70", "below_or_equal_70"]]

# Save to Excel
filename = f'spyrometer_below07-{name_suffix}.xlsx'
final_df.to_excel(filename, index=False)

print(f"Results saved to {filename}")

Results saved to spyrometer_below07-All.xlsx


In [20]:
# Define the filename
#save dataframe
filename = f'spyrometer_output-{name_suffix}.xlsx'
# Get the current working directory
current_dir = os.getcwd()
file_path = os.path.join(current_dir, filename)


# DataFrames for each dataset
dataframes = {
    "Hepa 7": dfHepa7,
    "Hepa 5": dfHepa5,
    "APSU": dfAPSU,
    "APSN": dfAPSN
}

# Initialize an Excel writer
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    # Loop through each dataset and save it to a separate sheet
    for dataset_name, df in dataframes.items():
        # Save each DataFrame to its corresponding sheet
        df.to_excel(writer, sheet_name=dataset_name, index=False)

print(f"DataFrames saved to {file_path}")

DataFrames saved to C:\Users\ejnar\OneDrive\DTU\speciale_projekt\Spirometer fra test\spyrometer_output-All.xlsx
